In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=76643c2e5f8451d64b84396504cb9f8ff5eedd589b6d847104d6fd844f8cf84e
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import pyspark

In [3]:
import psycopg2
import pandas as pd
from pyspark.sql import SparkSession

In [4]:
conn= psycopg2.connect("dbname='mtamypvc' user='mtamypvc' host='rosie.db.elephantsql.com' password='A-3BMAj6OCp6QByyHRgo2UCqYhoXy-_d'")

In [5]:
db_params={
    "dbname":'mtamypvc',
    "user":'mtamypvc' ,
    "host":'rosie.db.elephantsql.com' ,
    "password":'A-3BMAj6OCp6QByyHRgo2UCqYhoXy-_d',
    "port":"5432"
}

In [6]:
connection=psycopg2.connect(**db_params)

In [7]:
cur=conn.cursor()

In [8]:
spark=SparkSession.builder.appName("Data Lake").getOrCreate()

In [9]:
from pyspark.sql.types import StructType,StringType,StructField

def fetch_table_as_df(table_name):
    cur.execute(f"SELECT * FROM {table_name}")
    columns = [desc[0] for desc in cur.description]
    # Define the schema explicitly
    schema = StructType([
        StructField(col_name, StringType(), nullable=True) for col_name in columns
    ])

    data = cur.fetchall()
    return spark.createDataFrame(data, schema)

In [10]:
def create_datalake(spark_df,table_name):
  spark_df.write.parquet(f"{table_name}.parquet",mode="overwrite")

In [11]:
table_names=[
    'employees','orders','order_details','customers','territories'
]

In [12]:
for tab_name in table_names:
  df=fetch_table_as_df(tab_name)
  create_datalake(df,tab_name)


In [13]:
pwd

'/content'

In [14]:
customers=spark.read.parquet('customers.parquet')

In [15]:
customers.createOrReplaceTempView('customers_temp')

In [16]:
spark.sql('select * from customers_temp').show()

+-----------+--------------------+-------------------+--------------------+--------------------+---------------+-------------+-----------+---------+---------------+---------------+
|customer_id|        company_name|       contact_name|       contact_title|             address|           city|       region|postal_code|  country|          phone|            fax|
+-----------+--------------------+-------------------+--------------------+--------------------+---------------+-------------+-----------+---------+---------------+---------------+
|      LILAS|   LILA-Supermercado|    Carlos González|  Accounting Manager|Carrera 52 con Av...|   Barquisimeto|         Lara|       3508|Venezuela|   (9) 331-6954|   (9) 331-7256|
|      LINOD|    LINO-Delicateses|   Felipe Izquierdo|               Owner|Ave. 5 de Mayo Po...|I. de Margarita|Nueva Esparta|       4980|Venezuela|   (8) 34-56-12|   (8) 34-93-93|
|      LONEP|Lonesome Pine Res...|        Fran Wilson|       Sales Manager|  89 Chiaroscuro Rd.